In [1]:
try:
    import xml.etree.cElementTree as ET
except ImportError:
    import xml.etree.ElementTree as ET

import numpy as np
import pandas as pd

# INPUT CONSTANTS
STEP_TAG = 'step3'
EXT_MODEL_TAG = 'stats'
PATH_STEP3 = '/home/jaum/PG/pg/Final_Output/step3/stats/'
# SCORE TAGS
SCORE_ACC_TAG = 'acc'#acc or f1
SCORE_F1_TAG = 'f1'
# CONSTANTS FROM CV CONFIG
CV_TAG = 'cv'
CV_TYPE_NORMAL_TAG = 'normal'
CV_TYPE_UNBIASED_TAG = 'unbiased'
CV_MODE_NESTED_TAG = 'nested'
# CLF TAGS
CLF_KNN_TAG = 'knn'
CLF_RF_TAG = 'rf'
CLF_SVM_TAG = 'svm'
#
XML_TAG = '.xml'
RESULTS_TAG = 'results'
XML_SCORE_TAG = 'mean_score'
#tags
NN_TAG = 'Normal_Nested_{0}'
UN_TAG = 'Unbiased_Nested_{0}'

In [2]:
def parse_unique_tag(file_in,tag,ret_attr):

    unique_tags = []
    for elem in file_in.iter(tag):
        return (float(elem.attrib[ret_attr]))

def get_classifier_files_path(clf_tag, score_tag):
    # files names
    nn_file = '{0}_{1}_{2}_{3}_{4}_{5}_{6}{7}'.format(STEP_TAG, EXT_MODEL_TAG, CV_TYPE_NORMAL_TAG,
                                                             CV_MODE_NESTED_TAG, clf_tag, score_tag, RESULTS_TAG, XML_TAG)
    un_file = '{0}_{1}_{2}_{3}_{4}_{5}_{6}{7}'.format(STEP_TAG, EXT_MODEL_TAG, CV_TYPE_UNBIASED_TAG,
                                                           CV_MODE_NESTED_TAG, clf_tag, score_tag, RESULTS_TAG, XML_TAG)
    # files path
    file_path = '{0}{1}/'.format(PATH_STEP3, score_tag)
    
    return file_path + nn_file, file_path + un_file

def get_classifier_scores(clf_tag, score_tag):
    files_path = get_classifier_files_path(clf_tag, score_tag)
    
    clf_scores = []
    for file_path in files_path:
        file_iter = ET.parse(file_path)
        clf_scores.append(parse_unique_tag(file_iter, CV_TAG, XML_SCORE_TAG))
    
    return clf_scores

def get_df(clf_tags, score_tag):
    ss_index = [NN_TAG, UN_TAG]
    ss_index = [tag.format(score_tag.title()) for tag in ss_index]
    
    clf_scores_ss = []
    for clf_tag in clf_tags:
        clf_scores = get_classifier_scores(clf_tag, score_tag)
        clf_scores_ss.append(pd.Series(clf_scores, index=ss_index))
    
    return pd.DataFrame([clf_ss for clf_ss in clf_scores_ss], index= ['Grid_KNN', 'Grid_RandomForest', 'Grid_SVM'])
    

In [3]:
# STEP3 Results for score ACCURACY using the STATISTICAL ANALISYS as feature extraction model
get_df([CLF_KNN_TAG, CLF_RF_TAG, CLF_SVM_TAG], SCORE_ACC_TAG)

,Normal_Nested_Acc,Unbiased_Nested_Acc
Grid_KNN,0.947353,0.805505
Grid_RandomForest,0.980062,0.810868
Grid_SVM,0.937272,0.727503


In [4]:
# STEP3 Results for score F1_MACRO using the STATISTICAL ANALISYS as feature extraction model
get_df([CLF_KNN_TAG, CLF_RF_TAG, CLF_SVM_TAG], SCORE_F1_TAG)

,Normal_Nested_F1,Unbiased_Nested_F1
Grid_KNN,0.934902,0.522305
Grid_RandomForest,0.974669,0.568306
Grid_SVM,0.929567,0.445932
